# Tutorial on Sentiment Analyis using BERT

In [1]:
!pip install transformers datasets
!pip install evaluate
!pip install accelerate>=0.20.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 989.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 2.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.9/439.9 kB 3.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 3.8 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.3/296.3 kB 3.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 3.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.9/25.9 MB 2.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 1.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 369.9/369.9 kB 3.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━

zsh:1: 0.20.1 not found


In [3]:
import pandas as pd
import datasets
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [7]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [4]:
data_train = pd.read_csv('/kaggle/input/twitter/twitter_training.csv', header=None)
data_test = pd.read_csv('/kaggle/input/twitter/twitter_validation.csv', header=None)

data_train.rename(columns = {3:'text', 2: 'label'}, inplace = True)
data_test.rename(columns = {3:'text', 2: 'label'}, inplace = True)

# data_train = data_train[data_train['label'] != 'Irrelevant']
# data_test = data_test[data_test['label'] != 'Irrelevant']

In [5]:
import re

#Text transformation
data_train['text']=data_train.text.str.lower()
data_train['text']=[str(data) for data in data_train.text]
data_train['text']=data_train.text.apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x))


data_test["text"]=data_test.text.str.lower() #lowercase
data_test["text"]=[str(data) for data in data_test.text] #converting all to string
data_test["text"]=data_test.text.apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x)) #regex

In [6]:
data_train

,0,1,label,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,i am coming to the borders and i will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,just realized that the windows partition of my...
74678,9200,Nvidia,Positive,just realized that my mac window partition is ...
74679,9200,Nvidia,Positive,just realized the windows partition of my mac ...
74680,9200,Nvidia,Positive,just realized between the windows partition of...


In [7]:
data_test

,0,1,label,text
0,3364,Facebook,Irrelevant,i mentioned on facebook that i was struggling ...
1,352,Amazon,Neutral,bbc news amazon boss jeff bezos rejects clai...
2,8312,Microsoft,Negative,microsoft why do i pay for word when it funct...
3,4371,CS-GO,Negative,csgo matchmaking is so full of closet hacking ...
4,4433,Google,Neutral,now the president is slapping americans in the...
...,...,...,...,...
995,4891,GrandTheftAuto(GTA),Irrelevant,toronto is the arts and culture capital of c...
996,4359,CS-GO,Irrelevant,this is actually a good move tot bring more vi...
997,2652,Borderlands,Positive,today sucked so it s time to drink wine n play...
998,8069,Microsoft,Positive,bought a fraction of microsoft today small wins


In [8]:
data_train = data_train[['text', 'label']]
data_test = data_test[['text', 'label']]
# data.rename(columns = {'tags': 'label'}, inplace=True)

In [9]:
# data_train = data_train.sample(frac = 1)

# # data_train = data_train.iloc[:20000]

In [10]:
data_train

,text,label
0,im getting on borderlands and i will murder yo...,Positive
1,i am coming to the borders and i will kill you...,Positive
2,im getting on borderlands and i will kill you ...,Positive
3,im coming on borderlands and i will murder you...,Positive
4,im getting on borderlands 2 and i will murder ...,Positive
...,...,...
74677,just realized that the windows partition of my...,Positive
74678,just realized that my mac window partition is ...,Positive
74679,just realized the windows partition of my mac ...,Positive
74680,just realized between the windows partition of...,Positive


In [11]:
data_test

,text,label
0,i mentioned on facebook that i was struggling ...,Irrelevant
1,bbc news amazon boss jeff bezos rejects clai...,Neutral
2,microsoft why do i pay for word when it funct...,Negative
3,csgo matchmaking is so full of closet hacking ...,Negative
4,now the president is slapping americans in the...,Neutral
...,...,...
995,toronto is the arts and culture capital of c...,Irrelevant
996,this is actually a good move tot bring more vi...,Irrelevant
997,today sucked so it s time to drink wine n play...,Positive
998,bought a fraction of microsoft today small wins,Positive


In [12]:
labels = list(set(data_train['label']))
label_encoding = {}
for i in range(len(labels)):
  label_encoding[labels[i]] = i
label_encoding

{'Positive': 0, 'Irrelevant': 1, 'Negative': 2, 'Neutral': 3}

In [13]:
rev_label_encoding = {v: k for k, v in label_encoding.items()}
rev_label_encoding

{0: 'Positive', 1: 'Irrelevant', 2: 'Negative', 3: 'Neutral'}

In [14]:
data_train = data_train.replace({'label': label_encoding})
data_test = data_test.replace({'label': label_encoding})

In [15]:
data_train

,text,label
0,im getting on borderlands and i will murder yo...,0
1,i am coming to the borders and i will kill you...,0
2,im getting on borderlands and i will kill you ...,0
3,im coming on borderlands and i will murder you...,0
4,im getting on borderlands 2 and i will murder ...,0
...,...,...
74677,just realized that the windows partition of my...,0
74678,just realized that my mac window partition is ...,0
74679,just realized the windows partition of my mac ...,0
74680,just realized between the windows partition of...,0


In [16]:
data_test

,text,label
0,i mentioned on facebook that i was struggling ...,1
1,bbc news amazon boss jeff bezos rejects clai...,3
2,microsoft why do i pay for word when it funct...,2
3,csgo matchmaking is so full of closet hacking ...,2
4,now the president is slapping americans in the...,3
...,...,...
995,toronto is the arts and culture capital of c...,1
996,this is actually a good move tot bring more vi...,1
997,today sucked so it s time to drink wine n play...,0
998,bought a fraction of microsoft today small wins,0


In [17]:
data_train =  datasets.Dataset.from_pandas(data_train)
data_test =  datasets.Dataset.from_pandas(data_test)
# data = data.remove_columns('__index_level_0__')

In [18]:
data_train

Dataset({
    features: ['text', 'label'],
    num_rows: 74682
})

In [19]:
data_test

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})

In [20]:
data_train[0]

{'text': 'im getting on borderlands and i will murder you all  ', 'label': 0}

In [21]:
# data = data.train_test_split(test_size=0.1)

In [22]:
data = datasets.DatasetDict()

data['train']= data_train
data['test'] = data_test


# data = data.remove_columns("__index_level_0__")


In [23]:
data['train'][34]

{'text': 'we finished borderlands 3 and finally yas  thank you everyone for hanging out everyone  it was fun  i will try to make stream tomorrow but if not i might make so some irl streams while awayu  10 we never shall see  thank you both so  much for the raids  mompou mumpow or  megamagwitch and 4  kfdmitch ',
 'label': 0}

In [24]:
tokaneizer_name = "bert-base-cased"
model_name = "bert-base-cased"

In [25]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(tokaneizer_name)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = data.map(tokenize_function, batched=True)

  0%|          | 0/75 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [26]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 74682
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [27]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(rev_label_encoding), id2label=rev_label_encoding, label2id=label_encoding, ignore_mismatched_sizes=True)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initi

In [28]:
metric1 = evaluate.load("f1")
metric2 = evaluate.load("accuracy")

In [29]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'f1': metric1.compute(predictions=predictions, references=labels, average='macro')['f1'], 'accuracy': metric2.compute(predictions=predictions, references=labels)['accuracy']}

In [30]:
training_args = TrainingArguments(output_dir="/kaggle/working/Bert-twttr-base",
                                  evaluation_strategy='epoch',
                                  save_strategy='epoch',
                                  num_train_epochs=1,
                                  per_device_train_batch_size=16,
                                  per_gpu_train_batch_size=16,
                                  per_device_eval_batch_size=8,
                                  per_gpu_eval_batch_size=8,
                                  save_total_limit = 5,
                                  metric_for_best_model = 'f1',
                                  load_best_model_at_end=True,
                                 learning_rate = 5e-5
                                 )


In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [32]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wan

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.462600,0.166111,0.952533,0.953000


Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


TrainOutput(global_step=4668, training_loss=0.7054446284956879, metrics={'train_runtime': 4104.751, 'train_samples_per_second': 18.194, 'train_steps_per_second': 1.137, 'total_flos': 1.965001268912947e+16, 'train_loss': 0.7054446284956879, 'epoch': 1.0})

In [33]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/Bert-twttr-base/checkpoint-4668", num_labels=len(rev_label_encoding), id2label=rev_label_encoding, label2id=label_encoding)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)



Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [34]:
text = ['This product is very beautiful']

nlp(text)

[{'label': 'Positive', 'score': 0.9038189053535461}]